In [13]:
### This will be the routine used to take a non-nested GBR regional contributor table and produce the summaries
### that we typically supply
### will eventually try to declare as much stuff up front as possible

import pandas as pd
import numpy as np

selectedConstituents = ['Sediment - Fine', 'N_Particulate']
#rawcontrib = pd.read_csv('P:\projects\RC7_Results\FZ_Baseline_RegContributorDataGrid_Rob.csv')
rawcontrib = pd.read_csv('P:\projects\RC8_ResultsSets\BU\RobsRuns\BU_RC8_BL_RegContributorDataGrid_AusGovSource_AusGovReg.csv')

rawcontrib['AnnLoadToStreamKG'] = rawcontrib['LoadToStream (kg)'].div(rawcontrib['Num_Days']).mul(365.25)
rawcontrib['AnnLoadToExportKG'] = rawcontrib['LoadToRegExport (kg)'].div(rawcontrib['Num_Days']).mul(365.25)
rawcontrib['AreaHA'] = rawcontrib['AreaM2'].div(10000)
rawcontrib.reset_index()
rawcontrib.head(5)

,Rep_Region,ModelElement,Constituent,FU,AreaM2,Process,LoadToStream (kg),LoadToRegExport (kg),RSDR,Num_Days,AnnLoadToStreamKG,AnnLoadToExportKG,AreaHA
0,Lower Burdekin,SC #1608,Atrazine,Water,4386667.446,Undefined,0.0,0.0,0.0,10227,0.0,0.0,438.666745
1,Lower Burdekin,SC #1608,Atrazine,Water,4386667.446,Hillslope surface soil,0.0,0.0,0.0,10227,0.0,0.0,438.666745
2,Lower Burdekin,SC #1608,Atrazine,Water,4386667.446,Hillslope sub-surface soil,0.0,0.0,0.0,10227,0.0,0.0,438.666745
3,Lower Burdekin,SC #1608,Atrazine,Water,4386667.446,Gully,0.0,0.0,0.0,10227,0.0,0.0,438.666745
4,Lower Burdekin,SC #1608,Atrazine,Water,4386667.446,Hillslope no source distinction,0.0,0.0,0.0,10227,0.0,0.0,438.666745


In [15]:
### Identify supply regs, will use for aggregated areas & supply loads
supplyRegs = rawcontrib['Rep_Region'].unique()
supplyRegs

array(['Lower Burdekin', 'East Burdekin', 'Don River', 'Upper Burdekin',
       'Burdekin', 'Belyando', 'Bowen Bogie', 'Ross', 'Cape Campaspe',
       'Black'], dtype=object)

In [16]:
### Now name the regoins that are export locations
exportRegs = ['Lower Burdekin', 'Don River', 'Ross', 'Black']

In [17]:
### 
catchFuArea = pd.DataFrame(rawcontrib.groupby(['ModelElement','FU']).agg({'AreaHA':'first'})).reset_index()
catchFuArea.head(10)

,ModelElement,FU,AreaHA
0,SC #1,Conservation,0.000000
1,SC #1,Dryland Cropping,0.000000
2,SC #1,Forestry,0.000000
3,SC #1,Grazing Forested,3047.473334
4,SC #1,Grazing Open,3161.070635
5,SC #1,Horticulture,0.000000
6,SC #1,Irrigated Cropping,0.000000
7,SC #1,Other,0.000000
8,SC #1,Stream,0.000000
9,SC #1,Sugarcane,0.000000


In [24]:
### subCat areas
catchArea = pd.DataFrame(catchFuArea.groupby(['ModelElement']).agg({'AreaHA':'sum'})).reset_index()
catchArea.head(8)

,ModelElement,AreaHA
0,SC #1,6425.01225
1,SC #10,13602.24000
2,SC #100,10241.99986
3,SC #1000,5959.53000
4,SC #1001,1781.46000
5,SC #1002,10181.97000
6,SC #1003,6706.89000
7,SC #1004,5679.99000


In [46]:
### RepRegion catchment composition
### needed something in the .agg for this to work
regCatchComp = pd.DataFrame(rawcontrib.groupby(['Rep_Region','ModelElement']).agg({'FU':'count'})).reset_index()
regCatchComp = regCatchComp.rename(columns={'ModelElement':'Catch'})
regCatchComp.head(8)

,Rep_Region,Catch,FU
0,Belyando,SC #1000,1118
1,Belyando,SC #1001,1118
2,Belyando,SC #1002,1118
3,Belyando,SC #1003,1118
4,Belyando,SC #1004,1118
5,Belyando,SC #1005,1118
6,Belyando,SC #1006,1118
7,Belyando,SC #1007,1118


In [47]:
### Catchment count for each reg??
### could use this to work which suuply reg each subcatch belongs to (lowest count of catchment of all relevant nested regs)
regCatchCount = pd.DataFrame(regCatchComp.groupby(['Rep_Region']).agg({'Catch':'count'})).reset_index()
regCatchCount = regCatchCount.rename(columns={'Rep_Region':'SupplyReg', 'Catch':'CatchCount'})
regCatchCount

,SupplyReg,CatchCount
0,Belyando,435
1,Black,17
2,Bowen Bogie,146
3,Burdekin,1374
4,Cape Campaspe,232
5,Don River,50
6,East Burdekin,1558
7,Lower Burdekin,1625
8,Ross,25
9,Upper Burdekin,474


In [48]:
### Catchment within what Regs (>1 indicates nested)
## probably don't need
catchInRegs = pd.DataFrame(regCatchComp.groupby(['Catch']).agg({'Rep_Region':'count'})).reset_index()
catchInRegs = catchInRegs.rename(columns={'Rep_Region':'RegCount'})
catchInRegs.head(8)

,Catch,RegCount
0,SC #1,1
1,SC #10,1
2,SC #100,3
3,SC #1000,4
4,SC #1001,4
5,SC #1002,4
6,SC #1003,4
7,SC #1004,4


In [52]:
#catchToRelevantRegs = pd.DataFrame(regCatchComp.groupby(['Catch', 'Rep_Region']).agg({'FU':'first'})).reset_index()
catchToRelevantRegs = pd.merge(pd.DataFrame(regCatchComp.groupby(['Catch', 'Rep_Region']).agg({'FU':'first'})).reset_index(), regCatchCount, how='left', left_on=['Rep_Region'], right_on = ['SupplyReg']).reset_index()
catchToRelevantRegs.head(12)

,index,Catch,Rep_Region,FU,SupplyReg,CatchCount
0,0,SC #1,Upper Burdekin,1118,Upper Burdekin,474
1,1,SC #10,Ross,1118,Ross,25
2,2,SC #100,Bowen Bogie,1118,Bowen Bogie,146
3,3,SC #100,East Burdekin,1118,East Burdekin,1558
4,4,SC #100,Lower Burdekin,1118,Lower Burdekin,1625
5,5,SC #1000,Belyando,1118,Belyando,435
6,6,SC #1000,Burdekin,1118,Burdekin,1374
7,7,SC #1000,East Burdekin,1118,East Burdekin,1558
8,8,SC #1000,Lower Burdekin,1118,Lower Burdekin,1625
9,9,SC #1001,Belyando,1118,Belyando,435


In [57]:
### This doesn't yield what I want....
#catchSupplyReg = pd.DataFrame(catchToRelevantRegs.groupby(['Catch', 'SupplyReg']).agg({'CatchCount':'min'})).reset_index()
#catchMinRegCount = pd.DataFrame(catchToRelevantRegs.groupby(['Catch']).agg({'CatchCount':'min'})).reset_index()
###df.loc[df.groupby("item")["diff"].idxmin()]
catchSupplyReg = catchToRelevantRegs.loc[catchToRelevantRegs.groupby(['Catch'])['CatchCount'].idxmin()]
catchSupplyReg.head(15)

,index,Catch,Rep_Region,FU,SupplyReg,CatchCount
0,0,SC #1,Upper Burdekin,1118,Upper Burdekin,474
1,1,SC #10,Ross,1118,Ross,25
2,2,SC #100,Bowen Bogie,1118,Bowen Bogie,146
5,5,SC #1000,Belyando,1118,Belyando,435
9,9,SC #1001,Belyando,1118,Belyando,435
13,13,SC #1002,Belyando,1118,Belyando,435
17,17,SC #1003,Belyando,1118,Belyando,435
21,21,SC #1004,Belyando,1118,Belyando,435
25,25,SC #1005,Belyando,1118,Belyando,435
29,29,SC #1006,Belyando,1118,Belyando,435


In [60]:
supplyRegCatchFuArea = pd.merge(catchSupplyReg.groupby(['SupplyReg','Catch']).agg({'CatchCount':'first'}).reset_index(), catchFuArea, how='left', left_on=['Catch'], right_on = ['ModelElement']).reset_index()
supplyRegCatchFuArea.head(16)

,index,SupplyReg,Catch,CatchCount,ModelElement,FU,AreaHA
0,0,Belyando,SC #1000,435,SC #1000,Conservation,0.000000
1,1,Belyando,SC #1000,435,SC #1000,Dryland Cropping,0.000000
2,2,Belyando,SC #1000,435,SC #1000,Forestry,0.000000
3,3,Belyando,SC #1000,435,SC #1000,Grazing Forested,281.958388
4,4,Belyando,SC #1000,435,SC #1000,Grazing Open,5674.781335
5,5,Belyando,SC #1000,435,SC #1000,Horticulture,0.000000
6,6,Belyando,SC #1000,435,SC #1000,Irrigated Cropping,0.000000
7,7,Belyando,SC #1000,435,SC #1000,Other,0.000000
8,8,Belyando,SC #1000,435,SC #1000,Stream,0.000000
9,9,Belyando,SC #1000,435,SC #1000,Sugarcane,0.000000


In [62]:
supplyRegFuArea = pd.DataFrame(supplyRegCatchFuArea.groupby(['SupplyReg','FU']).agg({'AreaHA':'sum'})).reset_index()
supplyRegFuArea.head(15)

,SupplyReg,FU,AreaHA
0,Belyando,Conservation,8.781773e+04
1,Belyando,Dryland Cropping,1.536290e+04
2,Belyando,Forestry,4.698551e+04
3,Belyando,Grazing Forested,8.991735e+05
4,Belyando,Grazing Open,2.478738e+06
5,Belyando,Horticulture,0.000000e+00
6,Belyando,Irrigated Cropping,1.314085e+03
7,Belyando,Other,5.452320e+02
8,Belyando,Stream,0.000000e+00
9,Belyando,Sugarcane,0.000000e+00


In [ ]:
### will need to join the catch & supply reg back to annualised contributor to get te aggregated supplies